In [21]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.mask
from matplotlib import pyplot as plt
from rasterio import Affine
from rasterio.warp import reproject, Resampling

In [31]:
#Process the county sowing dataset
current_year = 2000
current_year_str = str(current_year)
os.chdir('/home/heisenberg/my_repo/Data')

df = pd.read_csv('maize.alldat.nolatlon.csv')
IL = (df['state']=='IL')&(df['YEAR']==current_year)
IL_county = df[IL]
col_list = ['FIPS','sday']
IL_county = IL_county[col_list]
IL_county = IL_county.groupby(['FIPS']).mean()
sow_date = list(IL_county['sday'])
del df

#Get all county FIPS in Illinois
county_FIPS = IL_county.index.values
county_FIPS = county_FIPS.tolist()
county_FIPS = list(map(str,county_FIPS))

In [32]:
#Get all counties in IL in the shape file
os.chdir('/home/heisenberg/my_repo/Data/County_Shape')
county_shape = gpd.read_file('CoUS_GCS12.shp')
county_shape = county_shape[county_shape['atlas_stco'].isin(county_FIPS)]
cols = ['atlas_stco','geometry']
county_shape = county_shape[cols]

In [33]:
#Read CDL data and change the CDL data to corn/non-corn
os.chdir('/home/heisenberg/my_repo/Data/CDL')
cdl_name = 'CDL_' + current_year_str + '.tif'
cdl = rasterio.open(cdl_name)
cdl_data = cdl.read(1)
cdl_data[cdl_data != 1] = 0

#Read state shape file
os.chdir('/home/heisenberg/my_repo/Data')
state = gpd.read_file('cb_2017_us_state_500k/')

In [34]:
def mask_by_county(county_shape, corn, i):
    img_clip, trans = rasterio.mask.mask(corn, [county_shape.iloc[i].geometry], crop=True)
    img_clip = img_clip.astype('float32')
    img_clip[img_clip == 0] = np.nan
    index = np.nanmean(img_clip)
    return (img_clip[0,:,:],index)

In [35]:
#Function that returns GCVI index for each county for each GCVI GeoTIFF
def calculate_GCVI(filename, state, cdl, cdl_data, county_shape):
    gcvi = rasterio.open(filename)
    state.crs = gcvi.crs

    img_clip, trans = rasterio.mask.mask(gcvi, [state.iloc[1].geometry], crop=True)
    img_clip = img_clip.astype('float32')
    img_clip[img_clip == 0] = np.nan
    #plt.imshow(img_clip[0,:,:],vmin=0, vmax=800, cmap='Spectral')

    #Project the CDL to GCVI size
    newarr = np.empty(shape=(img_clip.shape[0],
                             img_clip.shape[1],img_clip.shape[2]))
    reproject(
        cdl_data, newarr,
        src_transform = cdl.transform,
        dst_transform = gcvi.transform,
        src_crs = cdl.crs,
        dst_crs = gcvi.crs,
        resampling = Resampling.average)

    #Bigger than 0.5 means there is corn, while smaller than 0.5 means no corn
    is_corn = newarr >= 0.5
    corn_field = is_corn.astype(int)
    corn_field = corn_field.astype(np.float64)

    #Finally mask the corn/non-corn matrix with gcvi to get gcvi indexes
    gcvi_corn = np.multiply(img_clip, corn_field)
    #plt.imshow(gcvi_corn.squeeze())

    #Re-Write the projected GCVI to a new tif file
    os.chdir('/home/heisenberg/my_repo/Data/Output/')
    new_dataset = rasterio.open('filter1.tif', 'w', driver='GTiff',
                                height = corn_field.shape[1], width = corn_field.shape[2],
                                count = 1, dtype = corn_field.dtype,
                                crs = gcvi.crs,
                                transform = gcvi.transform)

    new_dataset.write(gcvi_corn.squeeze(),1)
    new_dataset.close()

    #Change the projection of shape file to cdl file.
    county_shape.crs = cdl.crs
    num_of_county = len(county_shape.index)


    #Read saved gcvi_corn raster file for masking
    test = rasterio.open('filter1.tif')
    #corn_by_county = {}
    index_by_county = {}

    for county in range(num_of_county):
        result = mask_by_county(county_shape,test,county)
        #corn_by_county[county] = result[0]
        index_by_county[county] = result[1]
    os.remove('filter1.tif')
    return index_by_county

In [44]:
#Iterate every tif file in a specific year's gcvi directory.
gcvi_directory = os.fsdecode('/home/heisenberg/my_repo/Data/GCVI/'+ current_year_str)
index = 0
name_index = 2000055
All_GCVI = {}

# for file in os.listdir(gcvi_directory):
#     filename = os.fsdecode(file)
#     if filename.endswith(".tif"):
#         os.chdir(gcvi_directory)
#         print(filename)
#         All_GCVI[index] = calculate_GCVI(filename, state, cdl, cdl_data, county_shape)
#         index += 1
#         print("image"+str(index)+" completed \n")
#         continue
#     else:
#         print("Not a GCVI file")


while index < 200:
    os.chdir(gcvi_directory)
    filename = 'GCVI_filled_' + str(name_index) + '.tif'
    All_GCVI[index] = calculate_GCVI(filename, state, cdl, cdl_data, county_shape)
    index += 1
    name_index += 1
    print(filename)
    print("image "+str(index)+" completed \n")




 


GCVI_filled_2000055.tif
image 1 completed 

GCVI_filled_2000056.tif
image 2 completed 

GCVI_filled_2000057.tif
image 3 completed 

GCVI_filled_2000058.tif
image 4 completed 

GCVI_filled_2000059.tif
image 5 completed 

GCVI_filled_2000060.tif
image 6 completed 

GCVI_filled_2000061.tif
image 7 completed 

GCVI_filled_2000062.tif
image 8 completed 

GCVI_filled_2000063.tif
image 9 completed 

GCVI_filled_2000064.tif
image 10 completed 

GCVI_filled_2000065.tif
image 11 completed 

GCVI_filled_2000066.tif
image 12 completed 

GCVI_filled_2000067.tif
image 13 completed 

GCVI_filled_2000068.tif
image 14 completed 

GCVI_filled_2000069.tif
image 15 completed 

GCVI_filled_2000070.tif
image 16 completed 

GCVI_filled_2000071.tif
image 17 completed 

GCVI_filled_2000072.tif
image 18 completed 

GCVI_filled_2000073.tif
image 19 completed 

GCVI_filled_2000074.tif
image 20 completed 

GCVI_filled_2000075.tif
image 21 completed 

GCVI_filled_2000076.tif
image 22 completed 

GCVI_filled_2000077

GCVI_filled_2000236.tif
image 182 completed 

GCVI_filled_2000237.tif
image 183 completed 

GCVI_filled_2000238.tif
image 184 completed 

GCVI_filled_2000239.tif
image 185 completed 

GCVI_filled_2000240.tif
image 186 completed 

GCVI_filled_2000241.tif
image 187 completed 

GCVI_filled_2000242.tif
image 188 completed 

GCVI_filled_2000243.tif
image 189 completed 

GCVI_filled_2000244.tif
image 190 completed 

GCVI_filled_2000245.tif
image 191 completed 

GCVI_filled_2000246.tif
image 192 completed 

GCVI_filled_2000247.tif
image 193 completed 

GCVI_filled_2000248.tif
image 194 completed 

GCVI_filled_2000249.tif
image 195 completed 

GCVI_filled_2000250.tif
image 196 completed 

GCVI_filled_2000251.tif
image 197 completed 

GCVI_filled_2000252.tif
image 198 completed 

GCVI_filled_2000253.tif
image 199 completed 

GCVI_filled_2000254.tif
image 200 completed 



In [56]:
print(All_GCVI[0])


{0: 1.3349816, 1: 1.6361948, 2: 1.2003155, 3: 1.5464646, 4: 1.2401706, 5: 1.5394112, 6: 1.2194468, 7: 1.329478, 8: 1.1706338, 9: 1.4707632, 10: 2.89703, 11: 1.4050845, 12: 1.8004733, 13: 1.3992352, 14: 2.2125368, 15: 2.8431404, 16: 1.183224, 17: 1.244124, 18: 1.2321005, 19: 1.3163257, 20: 1.6066289, 21: 2.8355618, 22: 1.7185262, 23: 1.8010938, 24: 1.6523862, 25: 1.3404235, 26: 1.2728679, 27: 1.4814572, 28: 1.1490272, 29: 1.4516411, 30: 1.388071, 31: 1.2105951, 32: 1.4219397, 33: 2.9671752, 34: 2.2472656, 35: 1.6297513, 36: 1.3807253, 37: 1.2935039, 38: 1.2211603, 39: 2.1849785, 40: 1.2340595, 41: 1.3010045, 42: 1.288595, 43: 1.6283392, 44: 1.2113099, 45: 1.2253016, 46: 1.5088822, 47: 1.2949266, 48: 1.2019551, 49: 1.2553598, 50: 1.174377, 51: 1.9922464, 52: 1.5943009, 53: 2.0968702, 54: 1.6508465, 55: 1.4750693, 56: 1.4776604, 57: 1.4498669, 58: 1.5231491, 59: 1.7880248, 60: 1.6623756, 61: 1.1678569, 62: 1.1373285, 63: 1.1983864, 64: 1.8813664, 65: 1.0914803, 66: 1.3971491, 67: 1.467441